<a href="https://colab.research.google.com/github/Chaitanya1956/bages/blob/main/stockyyy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import random
import tensorflow as tf
from datetime import datetime, timedelta

# ======================
# 1. SETUP & CONFIGURATION
# ======================
random.seed(42)
np.random.seed(42)
tf.keras.utils.set_random_seed(42)

# ======================
# 2. USER INPUT
# ======================
ticker = input("Enter company ticker (e.g., AAPL, TSLA, MSFT): ").upper()
years = float(input("How many years of historical data? (1-5): "))

# ======================
# 3. FETCH LIVE DATA
# ======================
end_date = datetime.now().strftime('%Y-%m-%d')
start_date = (datetime.now() - timedelta(days=years*365)).strftime('%Y-%m-%d')

print(f"\nFetching {ticker} data from {start_date} to {end_date}...")
data = yf.download(ticker, start=start_date, end=end_date)
data = data[['Open', 'High', 'Low', 'Close', 'Volume']]

if len(data) < 60:
    raise ValueError(f"Only {len(data)} days available - need at least 60 trading days")

print(f"\nSuccess! Got {len(data)} trading days of data")
print(data.tail(3))

# ======================
# 4. PREPROCESSING
# ======================
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

# ======================
# 5. DYNAMIC LOOKBACK
# ======================
lookback = min(90, len(data) - 31)  # 3 months or max available
print(f"\nUsing {lookback} days lookback period")

# ======================
# 6. CREATE TRAINING DATA
# ======================
X, y = [], []
for i in range(lookback, len(scaled_data)):
    X.append(scaled_data[i-lookback:i])
    y.append(scaled_data[i, 3])  # Close price

X, y = np.array(X), np.array(y)

# ======================
# 7. BUILD MODEL
# ======================
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(X.shape[1], X.shape[2])),
    Dropout(0.2),
    LSTM(64),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')

# ======================
# 8. TRAIN
# ======================
print("\nTraining model...")
history = model.fit(X, y, epochs=30, batch_size=16, verbose=1)

# ======================
# 9. PREDICT
# ======================
last_sequence = scaled_data[-lookback:].reshape(1, lookback, -1)
prediction = model.predict(last_sequence, verbose=0)[0,0]

# Inverse transform
dummy = np.zeros((1, scaled_data.shape[1]))
dummy[0, 3] = prediction
predicted_price = scaler.inverse_transform(dummy)[0, 3]

# ======================
# 10. RESULTS
# ======================
current_price = float(data['Close'].iloc[-1])
change_pct = (predicted_price - current_price)/current_price * 100

print("\n" + "="*40)
print(f"{ticker} Stock Analysis")
print("="*40)
print(f"Current Price: ${current_price:.2f}")
print(f"Predicted Price: ${predicted_price:.2f} ({change_pct:+.2f}%)")

if change_pct > 2:
    print("\n🚀 Recommendation: BUY (Expected Growth)")
elif change_pct < -2:
    print("\n⚠️ Recommendation: SELL (Expected Decline)")
else:
    print("\n🔄 Recommendation: HOLD (Neutral Outlook)")

print("\nNote: Always consider other market factors")
print("="*40)

Enter company ticker (e.g., AAPL, TSLA, MSFT): AAPL
How many years of historical data? (1-5): 5


/tmp/ipython-input-4075129184.py:31: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed


Fetching AAPL data from 2020-08-13 to 2025-08-12...

Success! Got 1254 trading days of data
Price             Open        High         Low       Close     Volume
Ticker            AAPL        AAPL        AAPL        AAPL       AAPL
Date                                                                 
2025-08-07  218.631873  220.599641  216.334477  219.780563   90224800
2025-08-08  220.579666  230.738135  219.001455  229.090012  113854000
2025-08-11  227.919998  229.559998  224.759995  227.179993   61725300

Using 90 days lookback period

Training model...
Epoch 1/30



/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


73/73 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0296
Epoch 2/30
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0040
Epoch 3/30
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0038
Epoch 4/30
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0029
Epoch 5/30
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0033
Epoch 6/30
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0030
Epoch 7/30
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0029
Epoch 8/30
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0028
Epoch 9/30
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0027
Epoch 10/30
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0022
Epoch 11/30
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0025
Epoch 12/30
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0029
Epoch 13/30
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0023
Epoch 14/30
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0022
Epoch 15/30
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0022
Epoch 16/30
73/73 ━━━━━━━

/tmp/ipython-input-4075129184.py:95: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  current_price = float(data['Close'].iloc[-1])
